<a href="https://colab.research.google.com/github/Dezvi/Wave2d/blob/main/WaveEquation2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
!pip install pycuda
#Import packages we need
import numpy as np
from matplotlib import animation, rc, cm
from matplotlib import pyplot as plt

import pycuda
import pycuda.compiler as cuda_compiler
import pycuda.driver as cuda_driver
from pycuda.gpuarray import GPUArray

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.1 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1-cp310-cp310-linux_x86_64.whl size=661205 sha256=270bcf04a1a193e7b957fa01006cb2cc0f3d27875eaeb800b7f9f885eb338bf0
  Stored in directory: /root/.cache/pip/wheels/12/34/d2/9a349255a4eca3a486d82c79d21e138ce2ccd90f414d9d72b8
Successfully built pycuda


We import the same modules that we imported during the HeatMap2d.

In [8]:
class WaveEquation2DGPU(object):
    def __init__(self):
        pass

    def initialize(self, u1, u0, kappa, dx, dt, dy, block_width=8, block_height=8):
        self.kappa = np.float32(kappa)
        self.dx = np.float32(dx)
        self.dt = np.float32(dt)
        self.nx = np.int32(u0.shape[1]-2)

        self.dy = np.float32(dy)
        self.ny = np.int32(u0.shape[0]-2)

        self.block_size = (block_width, block_height, 1)
        self.grid_size = (int(np.ceil(self.nx / block_width)), int(np.ceil(self.ny / block_height)), 1)
        self.stream = cuda_driver.Stream()

        self.u1_g = GPUArray(u0.shape, u0.dtype)
        self.u0_g = GPUArray(u0.shape, u0.dtype)

        self.u2_g = GPUArray(u1.shape, u1.dtype)

        self.u0_g.set_async(u0, stream=self.stream)

        self.u1_g.set_async(u1, stream=self.stream)

        cuda_kernel = \
        """
        #define BLOCK_WIDTH {:d}
        #define BLOCK_HEIGHT {:d}
        """.format(block_width, block_height) \
        + \
        """
        __global__ void waveEq2d(float* u2, const float* u1, const float* u0, float kappa, float dx, float dt, int nx, float dy, int ny) {
            //Plus one to skip the "ghost cells"
            int x_blockdim = blockDim.x;

            int i = blockIdx.x*x_blockdim + threadIdx.x + 1;
            int tx = threadIdx.x + 1;

            int y_blockdim = blockDim.y;

            int j = blockIdx.y*y_blockdim + threadIdx.y + 1;
            int ty = threadIdx.y + 1;


            //First read into shared memory, including the local ghost cells / apron
            __shared__ float u1_shared[BLOCK_HEIGHT+2][BLOCK_WIDTH+2];
            for (int i = threadIdx.y; i < BLOCK_HEIGHT+2; i += y_blockdim) {
                int min_y = min(blockIdx.y * y_blockdim + i, ny);
                for (int j = threadIdx.x; j < BLOCK_WIDTH+2; j += x_blockdim) {
                    int min_x = min(blockIdx.x * x_blockdim + j, nx);
                    u1_shared[i][j] = u1[min_y * (nx + 2) + min_x];
                }
            }
            __syncthreads();

            //Then fix the boundary conditions
            //[0 | 1 2 3 4 | 5]
            if (i == 1) {
                u1_shared[ty][tx-1] = u1_shared[ty][tx];
            }
            else if (i == nx) {
                u1_shared[ty][tx+1] = u1_shared[ty][tx];
            }
            else if (j == 1) {
                u1_shared[ty-1][tx] = u1_shared[ty][tx];
            }
            else if (j == ny) {
                u1_shared[ty+1][tx] = u1_shared[ty][tx];
            }

            if (i >= 1 && i <= nx && j >= 1 && j <= ny) {
                u2[j*(nx+2) + i] = 2.0f*u1_shared[ty][tx] - u0[j*(nx+2)+i]
                    + kappa*dt/(dx*dx) * (u1_shared[ty][tx-1] - 2.0f*u1_shared[ty][tx] + u1_shared[ty][tx+1])
                    + kappa*dt/(dy*dy) * (u1_shared[ty-1][tx] - 2.0f*u1_shared[ty][tx] + u1_shared[ty+1][tx]);

            }
        }
        """
        self.module = cuda_compiler.SourceModule(cuda_kernel, \
                                            options=['--use_fast_math'])
        self.wave_eqn_kernel = self.module.get_function("waveEq2d");
        self.wave_eqn_kernel.prepare("PPPfffifi")



    def step(self):
        self.wave_eqn_kernel.prepared_async_call(self.grid_size, self.block_size, self.stream, \
                                                    self.u2_g.gpudata, self.u1_g.gpudata, self.u0_g.gpudata, \
                                                    self.kappa, self.dx, self.dt, self.nx, self.dy, self.ny)
        self.u0_g, self.u1_g, self.u2_g = self.u1_g, self.u2_g, self.u0_g


    def download(self):
        #self.u2 = cuda_driver.pagelocked_empty(u1.shape, u1.dtype)
        u1 = np.empty(self.u1_g.shape, self.u1_g.dtype)
        self.u1_g.get(u1)
        return u1


#Create test input data
kappa = 1.0
nx = 100
dx = 1.0

ny = 50
dy = 2.0

dt = 0.2*min(dx / (2.0*kappa), dy / (2.0*kappa))

u0 = np.zeros((ny, nx), dtype=np.float32)
u1 = np.zeros((ny, nx), dtype=np.float32)

for j in range(ny):
    for i in range(nx):
        x = (i - nx/2.0) * dx
        y = (j - ny/2.0) * dy
        if (np.sqrt(x**2 + y**2) < 10*min(dx, dy)):
            u0[j, i] = 10.0

import pycuda.autoinit

pycuda.tools.make_default_context()
simulator = WaveEquation2DGPU()
simulator.initialize(u1, u0, kappa, dx, dt, dy)

/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: stream in out-of-thread context could not be cleaned up
  globals().clear()
/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: device_allocation in out-of-thread context could not be cleaned up
  globals().clear()
/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: module in out-of-thread context could not be cleaned up
  globals().clear()


We need to add an u2 array, and replace the calculation on u0 for u1. Then we have to update the calculations for u2 using u0 and u1. The rest of the code remains the same outside of the initialization of u1 and dt.

In [9]:
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111)

heat2d = plt.imshow(u0, extent=[0, nx*dx, 0, ny*dy])
plt.colorbar()

def animate(i):
    print(".", end='', flush=True)
    if (i > 0):
        for k in range(10):
            simulator.step()

    u2 = simulator.download()

    heat2d.set_data(u2)
    print(np.sum(u2[1:-1]))

plt.rcParams["animation.html"] = "html5"
anim = animation.FuncAnimation(fig, animate, range(50), interval=100)
plt.close()
anim

.1470.0
.1470.0
.1469.9996
.1469.999
.1469.9984
.1469.9976
.1469.9968
.1469.9957
.1469.9946
.1469.9933
.1469.9918
.1469.9901
.1469.988
.1469.9858
.1469.9833
.1469.9805
.1469.9777
.1469.9746
.1469.9584
.1469.6416
.1466.282
.1450.0396
.1412.6451
.1362.9806
.1307.4115
.1250.8784
.1203.0662
.1170.9856
.1150.8777
.1130.4097
.1112.5529
.1097.358
.1082.2634
.1069.4824
.1056.6307
.1045.3741
.1034.2202
.1024.0863
.1014.235
.1004.99316
.996.16394
.987.662
.979.6389
.971.6997
.964.00336
.955.2844
.943.8258
.923.712
.885.66425
.821.5675
.741.9929
